In [1]:
import mordred as md
import numpy as np
import optuna
import pandas as pd
import torch
import torch.nn as nn
import torch_geometric.nn as gnn
from catboost import CatBoostRegressor, Pool
from rdkit import Chem
from rdkit.Chem import MolFromSmiles, MolToSmiles
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.loader import DataLoader as GDataLoader
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map

from yellow_cards_workflow import BASE_DIR, MODEL_NAMES, OUTPUT_DIR, PROCESSED_DIR, SEED
from yellow_cards_workflow.datasets import (
    FCD_Dataset,
    FCFP_Dataset,
    GNNIMDataset,
    get_dict_gnn_dataset,
    prepare_gnn_dataset,
    CNN_Dataset,
)
from yellow_cards_workflow.preprocessing import get_clean_dataset, load_processed_data
from yellow_cards_workflow.utils import (
    encode_smiles,
    generate_fingerprints,
    generate_rdkit_descriptors,
)
from yellow_cards_workflow.models import CNN, GNN, FCD, FCFP

In [2]:
MOD_RATE = 2.5
MU = 0.0075

## Loading Data


In [ ]:
properties_df = pd.read_csv(
    BASE_DIR / f"data/processed/qm9-variable-mu-{MU}-{MOD_RATE}.csv",
    sep=";",
    index_col=0,
)
properties_df

In [4]:
supplier = Chem.SDMolSupplier(BASE_DIR / "data/input/gdb9.sdf")


In [5]:
molecules = np.array([x for x in tqdm(supplier)])
print(np.count_nonzero(molecules))

  0%|          | 0/133885 [00:00<?, ?it/s]

[15:41:16] Explicit valence for atom # 1 C, 5, is greater than permitted
[15:41:16] ERROR: Could not sanitize molecule ending on line 9097
[15:41:16] ERROR: Explicit valence for atom # 1 C, 5, is greater than permitted
[15:41:16] Explicit valence for atom # 1 C, 5, is greater than permitted
[15:41:16] ERROR: Could not sanitize molecule ending on line 35785
[15:41:16] ERROR: Explicit valence for atom # 1 C, 5, is greater than permitted
[15:41:16] Explicit valence for atom # 4 C, 5, is greater than permitted
[15:41:16] ERROR: Could not sanitize molecule ending on line 62866
[15:41:16] ERROR: Explicit valence for atom # 4 C, 5, is greater than permitted
[15:41:16] Explicit valence for atom # 2 C, 5, is greater than permitted
[15:41:16] ERROR: Could not sanitize molecule ending on line 66832
[15:41:16] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[15:41:16] Explicit valence for atom # 2 C, 5, is greater than permitted
[15:41:16] ERROR: Could not sanitize molecule en

132737


[15:41:18] Explicit valence for atom # 1 C, 4, is greater than permitted
[15:41:18] ERROR: Could not sanitize molecule ending on line 1027576
[15:41:18] ERROR: Explicit valence for atom # 1 C, 4, is greater than permitted
[15:41:18] Explicit valence for atom # 1 C, 4, is greater than permitted
[15:41:18] ERROR: Could not sanitize molecule ending on line 1027617
[15:41:18] ERROR: Explicit valence for atom # 1 C, 4, is greater than permitted
[15:41:18] Explicit valence for atom # 1 C, 4, is greater than permitted
[15:41:18] ERROR: Could not sanitize molecule ending on line 1027818
[15:41:18] ERROR: Explicit valence for atom # 1 C, 4, is greater than permitted
[15:41:18] Explicit valence for atom # 1 C, 4, is greater than permitted
[15:41:18] ERROR: Could not sanitize molecule ending on line 1027857
[15:41:18] ERROR: Explicit valence for atom # 1 C, 4, is greater than permitted
[15:41:18] Explicit valence for atom # 1 C, 4, is greater than permitted
[15:41:18] ERROR: Could not sanitize mo

In [6]:
clean_mask = np.load(BASE_DIR / "data/processed/qm9-mask.npy")
molecules = molecules[clean_mask]

In [7]:
noniso_smiles = np.array(
    list(
        map(
            lambda x: Chem.MolToSmiles(x, isomericSmiles=False, canonical=True),
            tqdm(molecules),
        )
    )
)
smiles = np.array(
    list(
        map(
            lambda x: Chem.MolToSmiles(x, isomericSmiles=True, canonical=True),
            tqdm(molecules),
        )
    )
)
len(np.unique(noniso_smiles)), len(np.unique(smiles))

  0%|          | 0/132398 [00:00<?, ?it/s]

  0%|          | 0/132398 [00:00<?, ?it/s]

(132398, 132398)

In [8]:
molecular_properties = properties_df["g298_atom"].to_numpy() / (-1000)
len(molecular_properties)

132398

In [9]:
morgan_fingerprints, rdkit_fingerprints = generate_fingerprints(molecules)

Generating Morgan fingerprints


  0%|          | 0/132398 [00:00<?, ?it/s]

Generating RDKit fingerprints


  0%|          | 0/132398 [00:01<?, ?it/s]

In [10]:
rdkit_descriptors = generate_rdkit_descriptors(molecules)

Loading / Generating RDKit descriptors


  0%|          | 0/132398 [00:00<?, ?it/s]

(132398, 217)
Scaling RDKit descriptors to zero mean and unit variance


In [9]:
smiles_dict, encoded_smiles = encode_smiles(smiles)

  0%|          | 0/132398 [00:00<?, ?it/s]

In [10]:
len(smiles_dict)+1

27

In [12]:
gnn_num_fingerprints, gnn_fingerprints, gnn_mol_bonds = prepare_gnn_dataset(molecules)

  0%|          | 0/132398 [00:00<?, ?it/s]

Atom Dict 0
Hybdn Dict 0
FPs Dict 1980


## 5Fold Split


In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

In [14]:
SEED = 42

In [15]:
def train_fn(model, optim, loss_fn, epochs, train_dl, eval_dl, name):
    writer = SummaryWriter(log_dir=BASE_DIR / "logs" / name, flush_secs=15)

    torch.cuda.empty_cache()
    b_train = 1e5
    b_eval = 1e5

    bar = tqdm(range(epochs), leave=False, position=1)
    for epoch in bar:
        epoch_train_loss = model.train_fn(optim, loss_fn, train_dl)
        b_train = min(b_train, epoch_train_loss)

        epoch_eval_loss = model.eval_fn(loss_fn, eval_dl)
        b_eval = min(b_eval, epoch_eval_loss)

        bar.set_postfix_str(
            f"{epoch_train_loss:.3f}({b_train:.3f}) | {epoch_eval_loss:.3f}({b_eval:.3f})"
        )

        writer.add_scalar("loss/train", epoch_train_loss, epoch)
        writer.add_scalar("loss/eval", epoch_eval_loss, epoch)
        if epoch_eval_loss <= b_eval:
            torch.save(model.state_dict(), BASE_DIR / f"models/{name}.pth")
    bar.close()
    return b_train, b_eval

In [16]:
kf = KFold(5, shuffle=True, random_state=SEED)
predicted_values = {
    "smiles": [],
    "values": [],
    "cnn": [],
    "gnn": [],
    "fcfp": [],
    "fcd": [],
    "cb": [],
}
pbar = tqdm(kf.split(np.arange(len(smiles))), position=0, total=5)
for k, (unique_train_idx, unique_eval_idx) in enumerate(pbar):
    train_gnn_fingerprints, eval_gnn_fingerprints = (
        [gnn_fingerprints[i] for i in unique_train_idx],
        [gnn_fingerprints[i] for i in unique_eval_idx],
    )
    train_gnn_mol_bonds, eval_gnn_mol_bonds = (
        [gnn_mol_bonds[i] for i in unique_train_idx],
        [gnn_mol_bonds[i] for i in unique_eval_idx],
    )
    train_encoded_smiles, eval_encoded_smiles = (
        encoded_smiles[unique_train_idx],
        encoded_smiles[unique_eval_idx],
    )
    train_rdkit_descriptors, eval_rdkit_descriptors = (
        rdkit_descriptors[unique_train_idx],
        rdkit_descriptors[unique_eval_idx],
    )
    train_morgan_fingerprints, eval_morgan_fingerprints = (
        morgan_fingerprints[unique_train_idx],
        morgan_fingerprints[unique_eval_idx],
    )
    train_rdkit_fingerprints, eval_rdkit_fingerprints = (
        rdkit_fingerprints[unique_train_idx],
        rdkit_fingerprints[unique_eval_idx],
    )
    train_molecular_properties, eval_molecular_properties = (
        molecular_properties[unique_train_idx],
        molecular_properties[unique_eval_idx],
    )

    predicted_values["smiles"].extend(smiles[unique_eval_idx])
    predicted_values["values"].extend(eval_molecular_properties)

    # pbar.write("Split Finished")
    model = CNN(
        len(smiles_dict) + 1,
        n_conv_layers=3,
        kernel_size=5,
        conv_channels=512,
        n_lin_layers=3,
    ).to(device)
    eval_ds = CNN_Dataset(eval_encoded_smiles, eval_molecular_properties)
    eval_dl = DataLoader(
        eval_ds, batch_size=512, shuffle=False, num_workers=4, persistent_workers=True
    )
    train_ds = CNN_Dataset(train_encoded_smiles, train_molecular_properties)
    train_dl = DataLoader(
        train_ds, batch_size=512, shuffle=True, num_workers=4, persistent_workers=True
    )
    # pbar.write("CNN Loaded")
    b_train, b_eval = train_fn(
        model=model,
        optim=torch.optim.Adam(model.parameters(), lr=1e-4),
        loss_fn=nn.L1Loss(reduction="sum"),
        epochs=200,
        train_dl=train_dl,
        eval_dl=eval_dl,
        name=f"5fold-variable-QM9-{MOD_RATE}-{MU}-cnn-{SEED}-{k}",
    )
    pbar.write(f"5Fold\t CNN\t {k}\t {b_train:.4f}\t\t {b_eval:.4f}")
    model.load_state_dict(
        torch.load(
            BASE_DIR / f"models/5fold-variable-QM9-{MOD_RATE}-{MU}-cnn-{SEED}-{k}.pth",
            map_location=device,
            weights_only=True,
        )
    )
    predicted_values["cnn"].extend(
        model.eval_fn(nn.L1Loss(reduction="sum"), eval_dl, return_predictions=True)
    )

    train_gnn_dataset = GNNIMDataset(
        get_dict_gnn_dataset(
            train_gnn_fingerprints, train_gnn_mol_bonds, train_molecular_properties
        )
    )
    eval_gnn_dataset = GNNIMDataset(
        get_dict_gnn_dataset(
            eval_gnn_fingerprints, eval_gnn_mol_bonds, eval_molecular_properties
        )
    )
    model = GNN(
        gnn_num_fingerprints,
        embed_fingerprints=16,
        n_conv_layers=9,
        conv_channels=1024,
        n_lin_layers=10,
    ).to(device)
    eval_dl = GDataLoader(
        eval_gnn_dataset,
        batch_size=128,
        shuffle=False,
        num_workers=4,
        persistent_workers=True,
    )
    train_dl = GDataLoader(
        train_gnn_dataset,
        batch_size=128,
        shuffle=True,
        num_workers=4,
        persistent_workers=True,
    )
    b_train, b_eval = train_fn(
        model=model,
        optim=torch.optim.Adam(model.parameters(), lr=1e-4),
        loss_fn=nn.L1Loss(reduction="sum"),
        epochs=200,
        train_dl=train_dl,
        eval_dl=eval_dl,
        name=f"5fold-variable-QM9-{MOD_RATE}-{MU}-gnn-{SEED}-{k}",
    )

    pbar.write(f"5Fold\t GNN\t {k}\t {b_train:.4f}\t\t {b_eval:.4f}")
    model.load_state_dict(
        torch.load(
            BASE_DIR / f"models/5fold-variable-QM9-{MOD_RATE}-{MU}-gnn-{SEED}-{k}.pth",
            map_location=device,
            weights_only=True,
        )
    )
    predicted_values["gnn"].extend(
        model.eval_fn(nn.L1Loss(reduction="sum"), eval_dl, return_predictions=True)
    )

    model = FCFP(n_layers=7, hidden_wts=4096).to(device)
    eval_ds = FCFP_Dataset(
        eval_morgan_fingerprints, eval_rdkit_fingerprints, eval_molecular_properties
    )
    eval_dl = DataLoader(
        eval_ds, batch_size=512, shuffle=False, num_workers=4, persistent_workers=True
    )
    train_ds = FCFP_Dataset(
        train_morgan_fingerprints, train_rdkit_fingerprints, train_molecular_properties
    )
    train_dl = DataLoader(
        train_ds, batch_size=512, shuffle=True, num_workers=4, persistent_workers=True
    )
    b_train, b_eval = train_fn(
        model=model,
        optim=torch.optim.Adam(model.parameters(), lr=1e-4),
        loss_fn=nn.L1Loss(reduction="sum"),
        epochs=250,
        train_dl=train_dl,
        eval_dl=eval_dl,
        name=f"5fold-variable-QM9-{MOD_RATE}-{MU}-fcfp-{SEED}-{k}",
    )
    pbar.write(f"5Fold\t FCFP\t {k}\t {b_train:.4f}\t\t {b_eval:.4f}")
    model.load_state_dict(
        torch.load(
            BASE_DIR / f"models/5fold-variable-QM9-{MOD_RATE}-{MU}-fcfp-{SEED}-{k}.pth",
            map_location=device,
            weights_only=True,
        )
    )
    predicted_values["fcfp"].extend(
        model.eval_fn(nn.L1Loss(reduction="sum"), eval_dl, return_predictions=True)
    )

    model = FCD(n_layers=7, hidden_wts=4096).to(device)
    eval_ds = FCD_Dataset(eval_rdkit_descriptors, eval_molecular_properties)
    eval_dl = DataLoader(
        eval_ds, batch_size=512, shuffle=False, num_workers=4, persistent_workers=True
    )
    train_ds = FCD_Dataset(train_rdkit_descriptors, train_molecular_properties)
    train_dl = DataLoader(
        train_ds, batch_size=512, shuffle=True, num_workers=4, persistent_workers=True
    )
    b_train, b_eval = train_fn(
        model=model,
        optim=torch.optim.Adam(model.parameters(), lr=1e-4),
        loss_fn=nn.L1Loss(reduction="sum"),
        epochs=250,
        train_dl=train_dl,
        eval_dl=eval_dl,
        name=f"5fold-variable-QM9-{MOD_RATE}-{MU}-fcd-{SEED}-{k}",
    )
    pbar.write(f"5Fold\t FCD\t {k}\t {b_train:.4f}\t\t {b_eval:.4f}")
    model.load_state_dict(
        torch.load(
            BASE_DIR / f"models/5fold-variable-QM9-{MOD_RATE}-{MU}-fcd-{SEED}-{k}.pth",
            map_location=device,
            weights_only=True,
        )
    )
    predicted_values["fcd"].extend(
        model.eval_fn(nn.L1Loss(reduction="sum"), eval_dl, return_predictions=True)
    )

    eval_ds = Pool(
        np.hstack(
            [
                eval_morgan_fingerprints,
                eval_rdkit_fingerprints,
                eval_rdkit_descriptors,
            ]
        ),
        eval_molecular_properties,
    )
    trn_ds = Pool(
        np.hstack(
            [
                train_morgan_fingerprints,
                train_rdkit_fingerprints,
                train_rdkit_descriptors,
            ]
        ),
        train_molecular_properties,
    )
    model = CatBoostRegressor(
        loss_function="MAE",
        task_type="GPU",
        devices="0",
        metric_period=20,
        learning_rate=0.00116,
        depth=8,
        iterations=10000,
        use_best_model=True,
        silent=True,
        allow_writing_files=False,
    )
    model.fit(trn_ds, eval_set=eval_ds, plot=False)
    model.save_model(
        BASE_DIR / f"models/5fold-variable-QM9-{MOD_RATE}-{MU}-cb-{SEED}-{k}.cb"
    )
    b_train = model.get_best_score().get("learn", {"MAE": np.nan})["MAE"]
    b_eval = model.get_best_score().get("validation", {"MAE": np.nan})["MAE"]
    pbar.write(f"5Fold\t CB\t {k}\t {b_train:.4f}\t\t {b_eval:.4f}")
    model = CatBoostRegressor().load_model(
        BASE_DIR / f"models/5fold-variable-QM9-{MOD_RATE}-{MU}-cb-{SEED}-{k}.cb"
    )
    predicted_values["cb"].extend(np.stack(model.predict(eval_ds)))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

5Fold	 CNN	 0	 0.0043		 0.0037


  0%|          | 0/105918 [00:00<?, ?it/s]

  0%|          | 0/26480 [00:00<?, ?it/s]

...

5Fold	 GNN	 4	 0.0030		 0.0043


  0%|          | 0/250 [00:00<?, ?it/s]

5Fold	 FCFP	 4	 0.0021		 0.0049


  0%|          | 0/250 [00:00<?, ?it/s]

5Fold	 FCD	 4	 0.0024		 0.0028
5Fold	 CB	 4	 0.0023		 0.0044


In [17]:
predicted_df = pd.DataFrame(predicted_values)
predicted_df.to_csv(
    BASE_DIR / f"data/processed/qm9-variable-mu-predictions-{MU}-{MOD_RATE}-{SEED}.csv",
    sep=";",
)